In [ ]:
import cv2
import numpy as np
import os

import mediapipe as mp

from keras_facenet import FaceNet
from scipy.spatial.distance import cosine

# Load embeddings
data = np.load("faces_embeddings_done_4classes_new.npz", allow_pickle=True)
X_known, Y_known = data['arr_0'], data['arr_1']


In [ ]:

print(mp.__version__)           # should say 0.10.15
print(dir(mp.tasks.components))            # should include 'python'


In [ ]:
# from mediapipe.tasks.python.vision.object_detector import ObjectDetector
#from mediapipe.tasks.vision.object_detector import ObjectDetector


In [ ]:
# from mediapipe.tasks import vision
# from mediapipe.tasks.python import BaseOptions
# from mediapipe.tasks.python.vision.object_detector import ObjectDetector
# #from mediapipe.tasks.pythvision.object_detector import ObjectDetectorOptions

# from mediapipe.tasks.python.components.processors import running_mode

In [ ]:
import time

In [ ]:

embedder = FaceNet()
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5)

In [ ]:
"""
try:
    face_img = img_rgb[y1:y2, x1:x2]
    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
        label, confidence = recognize(face_img) 
except Exception as e:
    print("Recognition error:", e)
person.add_name(label)
person.add_confidence(confidence)
"""

In [ ]:
class Person:
    def __init__(self, id, center, name = "Unknown", confidence = 0.0):
        self.id = id
        self.center = center
        self.name = name
        self.confidence = confidence
        

    def add_name(self, name):
        self.name = name

    def add_confidence(self, confidence):
        self.confidence = confidence

    def add_center(self, center):
        self.center = center
    
    def get_id(self):
        return self.id
    
    def get_center(self):
        return self.center
    
    def get_name(self):
        return self.name

    def get_confidence(self):
        return self.confidence
    

#maybe have a check distance function

In [ ]:
from utils import streaming_utils_Tut
streaming_utils_Tut.getStream(0)

cap = streaming_utils_Tut.getStream()

In [ ]:
#change to objects
# sets stire the ids and the filter we would do the same thing, go through the object's ids and delete if the id is not in the set

TrackedPeople = []     # List of dicts tracking face info
# list of dicts: [{'id': 0, 'center': (x, y), 'label': "Jordan", 'confidence': 0.92}]
next_id = 0
recognize_count = 0
# Distance threshold (in pixels) to consider same face
DIST_THRESH = 60
frame_count = 0
# finds the box for the face and finds the center of the face which is used to
# figure out if we are looking at the same person frame by frame without rerecognizing the person
def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)
# maybe have if runtime is above 100 ms, make new faces unknown
# check match will for each already known person, distance is calculated and compared to our threshold. 
def check_match(box, center, matched_id, img_rgb):
    global next_id
    x1, y1, x2, y2 = box
    cx, cy = center
    label = ""
    confidence = 0.0
    for person in TrackedPeople:
        #print(frame_count, person.get_name(), "1")
        old_cx, old_cy = person.get_center()
        dist = np.linalg.norm([cx - old_cx, cy - old_cy])
        label = "Detecting..."
        confidence = 0.0
        if dist < DIST_THRESH:
            matched_ids.add(person.get_id()) #add same id
            person.add_center((cx, cy)) #add new center
            if frame_count % 15 != 0:
                return True

            # What if new label is unknown
            face_img = img_rgb[y1:y2, x1:x2]
            label, confidence, match_index = recognize(face_img)
            person.add_name(label)
            person.add_confidence(confidence)
            return True
    # maybe issue with multiple faces popping up when I move too fast can be fixed with looking at num detection
    # if num, detections == 1 then maybe 
    face_img = img_rgb[y1:y2, x1:x2]
    label, confidence, match_index = recognize(face_img) 
    if match_index == -1:
        match_index = next_id
    person = Person(match_index, (cx, cy), label, confidence)
    TrackedPeople.append(person)
    matched_ids.add(match_index)
    next_id += 1
            
            
    return False

def recognize(face_img):
    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
        face_img = img_rgb[y1:y2, x1:x2]
        face_img = cv2.resize(face_img, (160, 160))
        embedding = embedder.embeddings(np.expand_dims(face_img, axis=0))[0]
        distances = [cosine(embedding, emb) for emb in X_known]
        min_dist = min(distances)
        confidence = 1 - min_dist
        if(confidence > 0.6):
            match_index = distances.index(min_dist)
            label = Y_known[match_index]
            return label, confidence, match_index

    return "Unknown", 0.0, -1
    

# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = face_detector.process(img_rgb)

        matched_ids = set()
        if results.detections:
            for detection in results.detections:
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                
                Match = check_match(box, center, matched_ids, img_rgb)
                

        # Remove people not seen this frame (e.g., left frame) 
        TrackedPeople = [p for p in TrackedPeople if p.get_id() in matched_ids]
        print("recog", recognize_count, frame_count)
        # Draw boxes and labels
        counter = 0
        for person in TrackedPeople:
            counter += 1
            print(counter)
            cx, cy = person.get_center()
            x1, y1 = cx - 100, cy - 100
            x2, y2 = cx + 100, cy + 100
            label = person.get_name()
            confidence = person.get_confidence()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        
        cv2.imshow("Real-Time Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#change to objects
# sets stire the ids and the filter we would do the same thing, go through the object's ids and delete if the id is not in the set

TrackedPeople = []     # List of dicts tracking face info
# list of dicts: [{'id': 0, 'center': (x, y), 'label': "Jordan", 'confidence': 0.92}]
next_id = 0
recognize_count = 0
# Distance threshold (in pixels) to consider same face
DIST_THRESH = 60
frame_count = 0
# finds the box for the face and finds the center of the face which is used to
# figure out if we are looking at the same person frame by frame without rerecognizing the person
def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)

# check match will for each already known person, distance is calculated and compared to our threshold. 
def check_match(box, center, matched_ids, next_id):
    x1, y1, x2, y2 = box
    cx, cy = center
    label = ""
    confidence = 0.0
    for person in TrackedPeople:
        #print(frame_count, person.get_name(), "1")
        old_cx, old_cy = person.get_center()
        matched = False
        re_recognize = True
        dist = np.linalg.norm([cx - old_cx, cy - old_cy])
        label = "Detecting..."
        confidence = 0.3
        if dist < DIST_THRESH:
            matched = True
            matched_ids.add(person.get_id()) #add same id
            person.add_center((cx, cy)) #add new center
            if frame_count % 15 != 0:
                re_recognize = False

        if matched == True and re_recognize == False:
            return True
            
        

        if matched == True and re_recognize == True:
            face_img = img_rgb[y1:y2, x1:x2]
            #if frame_count % 5 == 0:
            if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                label, confidence = recognize(face_img)
            person.add_name(label)
            person.add_confidence(confidence)
            return True

    face_img = img_rgb[y1:y2, x1:x2]
    #if frame_count % 5 == 0:
    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
        label, confidence = recognize(face_img)  
    person = Person(next_id, (cx, cy), label, confidence)
    TrackedPeople.append(person)
    matched_ids.add(next_id)
    next_id += 1
            
            
    return False
"""
def check_match(box, center, matched_ids):
    x1, y1, x2, y2 = box
    cx, cy = center
    
    for person in TrackedPeople:
        #print(frame_count, person.get_name(), "1")
        old_cx, old_cy = person.get_center()
                    
        dist = np.linalg.norm([cx - old_cx, cy - old_cy])
        label = "Detecting..."
        confidence = 0.3
        if dist < DIST_THRESH:
            
            matched_ids.add(person.get_id()) #add same id
            person.add_center((cx, cy)) #add new center
            if frame_count % 15 == 0:
                try:
                    face_img = img_rgb[y1:y2, x1:x2]
                    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                        label, confidence = recognize(face_img) 
                except Exception as e:
                    print("Recognition error:", e)
                person.add_name(label)
                person.add_confidence(confidence)
                
            return True
    return False
"""
def recognize(face_img):
    global recognize_count
    recognize_count += 1
    face_img = cv2.resize(face_img, (160, 160))
    embedding = embedder.embeddings(np.expand_dims(face_img, axis=0))[0]
    distances = [cosine(embedding, emb) for emb in X_known]
    min_dist = min(distances)
    confidence = 1 - min_dist
    if(confidence > 0.6):
        match_index = distances.index(min_dist)
        label = Y_known[match_index]
    else:
        label = "Unknown"
    return label, confidence
    

# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = face_detector.process(img_rgb)

        matched_ids = set()
        if results.detections:
            for detection in results.detections:
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                
                Match = check_match(box, center, matched_ids, next_id)
                
                """
                if not Match:
                    
                    label = "Detecting..."
                    confidence = 0.0
                    face_img = img_rgb[y1:y2, x1:x2]
                    #if frame_count % 5 == 0:
                    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                        label, confidence = recognize(face_img)
                    person = Person(next_id, (cx, cy), label, confidence)
                    TrackedPeople.append(person)
                    matched_ids.add(next_id)
                    next_id += 1
                """
        # Remove people not seen this frame (e.g., left frame) 
        TrackedPeople = [p for p in TrackedPeople if p.get_id() in matched_ids]
        print("recog", recognize_count, frame_count)
        # Draw boxes and labels
        counter = 0
        for person in TrackedPeople:
            
            cx, cy = person.get_center()
            x1, y1 = cx - 100, cy - 100
            x2, y2 = cx + 100, cy + 100
            label = person.get_name()
            confidence = person.get_confidence()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        
        cv2.imshow("Real-Time Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:


TrackedPeople = []     # List of dicts tracking face info
# list of dicts: [{'id': 0, 'center': (x, y), 'label': "Jordan", 'confidence': 0.92}]
next_id = 0

# Distance threshold (in pixels) to consider same face
DIST_THRESH = 50


def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)


# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = face_detector.process(img_rgb)

        matched_ids = set()
        if results.detections:
            for detection in results.detections:
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                matched = False
                for person in TrackedPeople:
                    old_cx, old_cy = person['center']
                    dist = np.linalg.norm([cx - old_cx, cy - old_cy])
                    if dist < DIST_THRESH:
                        matched = True
                        matched_ids.add(person['id'])

                        # Only re-recognize every 5 frames
                        if frame_count % 5 == 0:
                            try:
                                face_img = img_rgb[y1:y2, x1:x2]
                                if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                                    face_img = cv2.resize(face_img, (160, 160))
                                    embedding = embedder.embeddings(np.expand_dims(face_img, axis=0))[0]
                                    distances = [cosine(embedding, emb) for emb in X_known]
                                    min_dist = min(distances)
                                    match_index = distances.index(min_dist)
                                    label = Y_known[match_index] if min_dist < 0.6 else "Unknown"
                                    confidence = 1 - min_dist
                                    person['label'] = label
                                    person['confidence'] = confidence
                            except Exception as e:
                                print("Recognition error:", e)
                        person['center'] = (cx, cy)
                        break

                if not matched:
                    label = "Detecting..."
                    confidence = 0.0
                    if frame_count % 5 == 0:
                        try:
                            face_img = img_rgb[y1:y2, x1:x2]
                            if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                                face_img = cv2.resize(face_img, (160, 160))
                                embedding = embedder.embeddings(np.expand_dims(face_img, axis=0))[0]
                                distances = [cosine(embedding, emb) for emb in X_known]
                                min_dist = min(distances)
                                match_index = distances.index(min_dist)
                                label = Y_known[match_index] if min_dist < 0.6 else "Unknown"
                                confidence = 1 - min_dist
                        except Exception as e:
                            print("Recognition error:", e)

                    TrackedPeople.append({
                        'id': next_id,
                        'center': (cx, cy),
                        'label': label,
                        'confidence': confidence
                    })
                    matched_ids.add(next_id)
                    next_id += 1

        # Remove people not seen this frame (e.g., left frame)
        TrackedPeople = [p for p in TrackedPeople if p['id'] in matched_ids]

        # Draw boxes and labels
        for person in TrackedPeople:
            cx, cy = person['center']
            x1, y1 = cx - 100, cy - 100
            x2, y2 = cx + 100, cy + 100
            label = person['label']
            confidence = person['confidence']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Real-Time Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Initializing mp's Face detector
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        start_detect = time.time()
        results = face_detector.process(img_rgb) #running face detection on the image
        end_detect = time.time()
        detection_latency_ms = (end_detect - start_detect) * 1000
        print(f"Face detection latency: {detection_latency_ms:.2f} ms")
        if results.detections: # if there are any detections
            detection_counter = 0
            for detection in results.detections: 
                
                # gets bounding box, normalizes, and coverts to pixel coords
                box = detection.location_data.relative_bounding_box 
                h, w = img_rgb.shape[:2]
                x1 = int(box.xmin * w)
                y1 = int(box.ymin * h)
                x2 = int((box.xmin + box.width) * w)
                y2 = int((box.ymin + box.height) * h)
                face = img_rgb[y1:y2, x1:x2]
                # a safety check for badly cropped faces, might take out 
                if face.shape[0] > 0 and face.shape[1] > 0:
                    # if any errors occurs we basically skip that frame 
                    
                    if frame_count % 5 == 0:
                        try:
                            
                            face = cv2.resize(face, (160, 160))
                            #converts the image to 4D [1, 160, 160, 3] and feeds it into FaceNet
                            embedding = embedder.embeddings(np.expand_dims(face, axis=0))[0] # Returns the 128D embedding vector for this face
                            
                            # Match using cosine similarity
                            distances = [cosine(embedding, emb) for emb in X_known]
                            min_dist = min(distances)
                            match_index = distances.index(min_dist)
                            label = Y_known[match_index] if min_dist < 0.6 else "Unknown"
                            
                            confidence = 1 - min_dist
                            
                            People[detection_counter] = (label, confidence)
                        except Exception as e:
                            print("Error processing face:", e)
            
                    
                    # Draw box + label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    if People[detection_counter] is not None:
                        cv2.putText(frame, f"{People[detection_counter][0]} ({People[detection_counter][1]:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    else:
                        cv2.putText(frame, "Detecting...", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                detection_counter = detection_counter + 1
        
        cv2.imshow("Real-Time Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# ret : if frame is read secessfully 

# Facenet time is mid 60s ms
# mediaPipes face recognition is 7-11 ms

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
from scipy.spatial.distance import cosine

# --- Setup ---
cap = cv2.VideoCapture(0)
frame_count = 0

mp_face = mp.solutions.face_detection
embedder = ...        # <- Your FaceNet embedder object
X_known = [...]        # <- Your list of known embeddings
Y_known = [...]        # <- Corresponding labels

TrackedPeople = []     # List of dicts tracking face info
next_id = 0

# Distance threshold (in pixels) to consider same face
DIST_THRESH = 50


def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)


# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = face_detector.process(img_rgb)

        matched_ids = set()
        if results.detections:
            for detection in results.detections:
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                matched = False
                for person in TrackedPeople:
                    # comparing each known persons detection to
                    old_cx, old_cy = person['center']
                    dist = np.linalg.norm([cx - old_cx, cy - old_cy])
                    # if distance of box is close enough to call it the same box
                    if dist < DIST_THRESH:
                        matched = True
                        matched_ids.add(person['id'])

                        # Only re-recognize every 5 frames
                        if frame_count % 5 == 0:
                            try:
                                #re recognizes and adds to known label
                            except Exception as e:
                                print("Recognition error:", e)
                        # update center for person every frame
                        break
                # if distance is too far to call it the same box for any of our tracked faces
                if not matched:
                    '''
                    label = "Detecting..." # probably will take out 
                    confidence = 0.0
                    '''
                    if frame_count % 5 == 0:
                        try:
                            face_img = img_rgb[y1:y2, x1:x2]
                            if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                                # Re recognize that detection because its most likely a new face
                        except Exception as e:
                            print("Recognition error:", e)

                    TrackedPeople.append({ 
                        'id': next_id,
                        'center': (cx, cy),
                        'label': label,
                        'confidence': confidence
                    })
                    matched_ids.add(next_id)
                    next_id += 1

        # Remove people not seen this frame (e.g., left frame)
        TrackedPeople = [p for p in TrackedPeople if p['id'] in matched_ids]
        # for each p in the list, include p if ...
        # Draw boxes and labels
        for person in TrackedPeople:
            cx, cy = person['center'] 
            x1, y1 = cx - 50, cy - 50
            x2, y2 = cx + 50, cy + 50
            label = person['label'] 
            confidence = person['confidence']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Real-Time Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
